In [2]:
import numpy as np
import anndata as ad
import scanpy as sc
import pandas as pd
import matplotlib.pyplot as plt
import anndata
import seaborn as sns
import scvelo as scv 
from scipy.stats import zscore
from dtne import *

In [28]:
import numpy as np
import matplotlib.pyplot as plt
import scvelo as scv

# 假设数据已经加载到 adata 中
adata = filtered_DG_PC1

# 设置 slice 分类
slice_categories = ['NR1_2', 'NR2_2', 'DR1_2', 'DR2_2']

# Step 1: 分别展示每个 slice 的散点图，并设置对应 colorbar 的最大值和最小值
fig, axes = plt.subplots(2, 2, figsize=(10, 10))  # 更大的图尺寸以适应所有子图

for i, slice_category in enumerate(slice_categories):
    ax = axes[i//2, i%2]  # 获取当前子图的位置
    # 筛选当前 slice 的数据
    subset = adata[adata.obs['slice'] == slice_category, :]
    
    # 计算当前 slice 的 PC1 的最小值和最大值
    pc1_slice_min = subset.obs['PC1'].min()
    pc1_slice_max = subset.obs['PC1'].max()
    
    # 手动绘制散点图，设置 colorbar 范围为该 slice 的 PC1 最小值和最大值
    sc = ax.scatter(subset.obsm['stereo'][:, 0], subset.obsm['stereo'][:, 1], 
                    c=subset.obs['PC1'], cmap='viridis', vmin=pc1_slice_min, vmax=pc1_slice_max, s=5)
    ax.set_title(slice_category)
    
    # 获取 colorbar 对象并将其放置到右下角，缩小 colorbar
    cbar = fig.colorbar(sc, ax=ax, orientation='vertical', pad=0.05, shrink=0.6)
    cbar.set_label('PC1', fontsize=12)
    
    # 隐藏坐标轴
    ax.axis('off')  # 去掉坐标轴

# 调整子图布局
plt.tight_layout()

# 显示图形
#plt.show()
output_path = "F:/spatial/HIP/figures/HIP_DG_PC1.pdf"
plt.savefig(output_path, format='pdf', bbox_inches='tight')
plt.close()
print(f"PDF saved as {output_path}")

PDF saved as F:/spatial/HIP/figures/HIP_DG_PC1.pdf


In [30]:
sns.kdeplot(
    data=filtered_NR_DG_PC1.obs, x="digital_layer", y="PC1",
    fill=True, thresh=0, levels=10, cmap="mako",clip=((1, 8.0), (-4, 4)))
plt.text(0.95, 0.95, 'NR', ha='center', va='center', transform=plt.gca().transAxes, color='white', fontsize=12)
#plt.show()
output_path = "F:/spatial/HIP/figures/HIP_NR_DG_digital_layer&PC1.pdf"
plt.savefig(output_path, format='pdf', bbox_inches='tight')
plt.close()
print(f"PDF saved as {output_path}")

PDF saved as F:/spatial/HIP/figures/HIP_NR_DG_digital_layer&PC1.pdf


In [32]:
sns.kdeplot(
    data=filtered_DR_DG_PC1.obs, x="digital_layer", y="PC1",
    fill=True, thresh=0, levels=10, cmap="mako",clip=((1, 8.0), (-4, 4)))
plt.text(0.95, 0.95, 'DR', ha='center', va='center', transform=plt.gca().transAxes, color='white', fontsize=12)
#plt.show()
output_path = "F:/spatial/HIP/figures/HIP_DR_DG_digital_layer&PC1.pdf"
plt.savefig(output_path, format='pdf', bbox_inches='tight')
plt.close()
print(f"PDF saved as {output_path}")

PDF saved as F:/spatial/HIP/figures/HIP_DR_DG_digital_layer&PC1.pdf


In [47]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde

# 假设DG.obs中有 "PC1" 和 "condition" 列
data = DG.obs['PC1']
condition_list = DG.obs['condition'].unique()

# 创建一个空字典来保存每个condition的FWHM
fwhm_dict = {}

# 使用sns.displot来绘制KDE图，返回一个FacetGrid对象
g = sns.displot(DG.obs, x="PC1", hue="condition", kind="kde")

# 获取FacetGrid中的Axes对象，使用g.axes[0, 0]来访问第一个子图的Axes对象
axes = g.axes[0, 0]

# 设置偏移量初始值
offset = 0  # 偏移量（百分比）

# 计算每个condition的FWHM并添加到图中
for condition in condition_list:
    condition_data = DG.obs[DG.obs['condition'] == condition]['PC1']
    
    # 使用scipy的gaussian_kde计算KDE
    kde = gaussian_kde(condition_data, bw_method='silverman')  # 可以选择合适的带宽方法
    x = np.linspace(min(condition_data), max(condition_data), 1000)
    y = kde(x)
    
    # 找到最大值和半最大值
    max_value = np.max(y)
    half_max = max_value / 2
    
    # 找到半最大值的左右位置（即FWHM的边界）
    fwhm_indices = np.where(y >= half_max)[0]
    fwhm_left = x[fwhm_indices[0]]
    fwhm_right = x[fwhm_indices[-1]]
    
    # 计算FWHM
    fwhm = fwhm_right - fwhm_left
    
    # 将计算出的FWHM存入字典
    fwhm_dict[condition] = fwhm
    
    # 在图中添加FWHM标注
    axes.axvline(fwhm_left, color='gray', linestyle='--', linewidth=1)
    axes.axvline(fwhm_right, color='gray', linestyle='--', linewidth=1)
    
    # 获取y轴的范围
    ymin, ymax = axes.get_ylim()
    
    # 动态调整文本的纵坐标
    text_y_position = ymax * 0.9 - offset
    offset += 0.15  # 每次增加偏移量，避免重叠
    
    # 将文本的纵坐标设置为y轴的适当位置
    axes.text((fwhm_left + fwhm_right) / 2, text_y_position, f"FWHM: {fwhm:.2f}", 
              horizontalalignment='center', color='black')

# 将计算出的FWHM值赋值到DG.obs['FWHM']中
DG.obs['FWHM'] = DG.obs['condition'].map(fwhm_dict)

# 添加标题和标签
g.fig.suptitle('DG L2/3', fontsize=16)
g.set_axis_labels('PC1', 'Density')
#plt.show()

# 显示保存了FWHM的DataFrame前几行（可选）
print(DG.obs[['condition', 'FWHM']].head())
output_path = "F:/spatial/V1_integration/NR_V/figures/DG_FWHM.pdf"
plt.savefig(output_path, format='pdf', bbox_inches='tight')
plt.close()

print(f"PDF saved as {output_path}")

      condition     FWHM
81822        NR  1.66355
81823        NR  1.66355
81824        NR  1.66355
81826        NR  1.66355
81828        NR  1.66355
PDF saved as F:/spatial/V1_integration/NR_V/figures/DG_FWHM.pdf
